In [2]:
from bs4 import BeautifulSoup
import requests
import os
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import time
from selenium import webdriver


In [3]:
executable_path = {'executable_path':'c:\\Users\\tmlun\\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [10]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all elements that contain book information
articles = soup.find('div', class_='list_text')
news_title = articles.find("div", class_="content_title").text
news_p = articles.find("div", class_="article_teaser_body").text


print(news_title)
print("**********")
print(news_p)

NASA's Curiosity Rover Reaches Its 3,000th Day on Mars
**********
As the rover has continued to ascend Mount Sharp, it’s found distinctive benchlike rock formations.


In [11]:
url = 'https://www.jpl.nasa.gov/images/?search=&category=Mars'
browser.visit(url)



In [25]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
images = soup.find('img', class_='main_image')

# images = sidebar.find_all('img', class_='thumb')['src']

image_url_list = []

for image in images:

    image_url = category.find('src')
    image_url_list.append(image_url)

jpeg_url_list = ['https://www.jpl.nasa.gov/' + url for url in image_url_list]

# titles_and_urls = zip(category_list, book_url_list)
try:
    for image_url in image_url_list:
        browser.links.find_by_partial_text('news_title').click()

# https://mars.nasa.gov/system/news_items/list_view_images/8832_PIA24180-panorama-320.jpg

# featured_image_url = 

except ElementDoesNotExist:
    print("Scraping Complete")

In [ ]:
# med_abbreviations = ['BMR', 'BP', 'ECG', 'MRI', 'qid', 'WBC']

# # We can use the read_html function in Pandas to automatically scrape any tabular data from a page (tables).
# tables = pd.read_html(url)
# tables

In [ ]:
# #What we get in return is a list of dataframes for any tabular data that Pandas found.
# type(tables)

In [ ]:
# #We can slice off any of those dataframes that we want using normal indexing.
# df = tables[]
# df.columns = ['abb', 'full_name', 'other']
# df.head

In [ ]:
# # Cleanup of extra row
# del df['other']

In [ ]:
# # Drop all single header rows
# df.columns = df.columns.get_level_values(0)
# df = df.loc[df.Ref.str.startswith("[")]
# df.head()

In [ ]:
# # Drop the first row and set the index to the `abb` column
# ### BEGIN SOLUTION
# df = df.iloc[1:]
# df.set_index('abb', inplace=True)
# df.head()
# ### END SOLUTION

In [ ]:
# # Split column values into two separate columns
# columnsplit = df['City'].str.split(", ", expand=True)
# df = df.assign(City=columnsplit[0],State=columnsplit[1])
# df.head()

In [ ]:
# # Drop a column
# df = df.drop(['Ref'], axis=1)
# df.head()

In [ ]:
# # Reset an index
# df = df.reset_index(drop=True)
# df.head()

In [ ]:
#  df.loc[df.State=="New York"]

In [ ]:
#  # Loop through the list of medical abbreviations and print the abbreviation
# # along with the full description.
# # Use the DataFrame to perform the lookup.
# ### BEGIN SOLUTION
# for abb in med_abbreviations:
#     print(abb, df.loc[abb].full_name)
# ### END SOLUTION

In [ ]:
book_url_list

In [ ]:
browser.quit()

In [ ]:
# Pandas also had a to_html method that we can use to generate HTML tables from DataFrames.
html_table = df.to_html()
html_table

In [ ]:
# You may have to strip unwanted newlines to clean up the table.
html_table.replace('\n', '')   

In [ ]:
# You can also save the table directly to a file.
df.to_html('table.html')